# Aula TP - 22/Fev/2022

#### Grupo 8:
- Melânia Pereira
- Duarte Oliveira
- Paulo Pereira

## Parte I

### Pergunta 1.1

Pode ler-se no segundo princípio de Kerchoff, o seguinte: *"The system must not require secrecy and can be stolen by the enemy without causing trouble"*, o que defende que um sistema criptográfico não deve depender de sigilo ou segredo e que deve poder ser roubado pelo inimigo sem que isso tenha implicações na sua segurança.

Percebe-se, aqui, que realmente nos Princípios de Kerchoff de 1883 já era evidenciada esta necessidade de eliminar o secretismo como fator de segurança nestes sistemas e é, sem dúvida, bastante fácil de entender o porquê: o segredo não é, de todo, algo confiável; há sempre uma grande probabilidade de que este venha a ser descoberto e, tratando-se de um fator na segurança, esta estaria comprometida no momento em que o segredo fosse desvendado. Assim, é quase imediato perceber que este não pode ser um fator decisivo e do qual a segurança completa e total de um sistema criptográfico dependa.

No entanto, o segredo pode continuar a ser usado como fator adicional e pode ser uma maneira muito eficiente de reduzir as probabilidades de ataques quando usado com outras camadas de segurança. O poder do sisgilo não deve ser substimado, é um fator adicional que pode ser muito útil.

# Parte II

### Pergunta 1.1

In [ ]:
import re
import time
from pycipher import Caesar
from math import log10

INITIAL_KEY = 10

class ngram_score(object):
    def __init__(self,ngramfile,sep=' '):
        ''' load a file containing ngrams and counts, calculate log probabilities '''
        self.ngrams = {}
        f = open(ngramfile)
        for line in f:
            key,count = line.split(sep) 
            self.ngrams[key] = int(count)
        self.L = len(key)
        self.N = sum(self.ngrams.values())
        #calculate log probabilities
        for key in self.ngrams.keys():
            self.ngrams[key] = log10(float(self.ngrams[key])/self.N)
        self.floor = log10(0.01/self.N)

    def score(self,text):
        ''' compute the score of text '''
        score = 0
        ngrams = self.ngrams.__getitem__
        for i in range(len(text)-self.L+1):
            if text[i:i+self.L] in self.ngrams: score += ngrams(text[i:i+self.L])
            else: score += self.floor          
        return score
       

fitness = ngram_score('english_quadgrams.txt')

def break_caesar(ctext, nseq):
    ctext = re.sub('[^A-Z]','',ctext.upper())
    scores = []
    for i in range(26):
        for j in range(nseq):
            scores.append((fitness.score(Caesar(i+j).decipher(ctext)),i))
    return max(scores)


def cipher_and_brute_force(text, n):
    for i in range(n):  
        text = Caesar(INITIAL_KEY+i).encipher(text)

    #print(text)

    start = time.time()
    max_key = break_caesar(text,n)
    end = time.time()

    print('elapsed time for '+str(n)+' ciphers: '+str(end-start))
    print ('best candidate with key (a,b) = '+str(max_key[1])+':')
    print (Caesar(max_key[1]).decipher(text))
    print('evaluating '+ str(25*n)+' keys')
    print()


# text = input("Texto a cifrar: ")
text = "Hello! How are you? This is a text to be encrypted. I am using ceasar cipher many times sequentually. This is probably bad english, but i dont really care."
# n = int(input("Número de aplicações da cifra de Cesar: "))

for n in range(1,6):
    cipher_and_brute_force(text,n)



### Pergunta 2.1

O texto limpo correspondente ao texto cifrado "OXAO" não pode ser "DATA", porque, numa cifra por substituição mono-alfabética, cada letra tem apenas uma letra correspondente que é fixa nessa cifra, ou seja, se a letra "O" corresponde à letra "D" na primeira ocorrência, não poderia corresponder à letra "A" na segunda ocorrência.

### Pergunta 3.1

In [ ]:
import numpy as np
from sympy import Matrix 

CHAR_TABLE = {chr(i+97): i for i in range(0,26)} #tabela 'letra : número' associado para cifra
CHAR_TABLE_REV = dict(zip(CHAR_TABLE.values(), CHAR_TABLE.keys())) #tabela 'número : letra' para decifra


def program():
    plain_text = sanitize(input('Enter text to cipher: '))
    print("Plain Text: ")
    print(plain_text)
    print("\n\n\n")

    key = sanitize(input('Enter key: '))
    print("Key: ")
    print(key)

    cipher_text = encrypt(plain_text, key)
    print("\n\n\n")
    print("Cipher Text: ")
    print(cipher_text)
    print("\n\n\n")

    try:
        decrypted_text = decrypt(cipher_text, key)
        print("Decrypted Text: ")
        print(decrypted_text)
        print("\n\n\n")
    except:
        print("Inverse of Matrix Does Not Exist.")



def encrypt(plain_text, key):
    """
    Hill Cipher Encryption
    C = P*K MOD 26
    where 
    C = cipher text
    P = plain text
    K = key (matrix)
    """
    # encontrar o tamanho da matriz de acordo com o tamanho da chave
    matrix_size = findMatrixSize(len(key)) 
    # adicionar padding à chave e converter letras para números de acordo com a CHAR_TABLE
    padded_key = getPaddedKey(key, matrix_size) 
    # transformar a string da chave num array e depois numa matriz
    encoded_matrix = np.fromstring(padded_key,dtype=int, sep=' ').reshape(matrix_size,matrix_size)
    # adicionar padding ao texto a cifrar, caso o seu tamanho não seja compativel com o tamanho da matriz para fazer a multiplicação
    # e converter letras para  números de acordo com o CHAR_TABLE
    padded_pt = getPaddedPlainText(plain_text, matrix_size)
    # transformar o texto a cifrar já convertido em números num array e depois numa matriz
    text_matrix = np.fromstring(padded_pt, dtype=int,sep=' ').reshape(-1, matrix_size)
    
    cipher_text = ''
    for column in text_matrix:
        # fazer a multiplicação da matriz 'chave' pelas colunas do texto a cifrar seguido do calculo do resto da divisão por 26
        x = (encoded_matrix @ column) % 26
        # para cada elemento resultante do calculo anterior, converter o número em letra de acordo com o CHAR_TABLE_REV
        for elem in x:
            cipher_text += CHAR_TABLE_REV[elem]
        # neste momento, cipher_text contém o texto cifrado
    return cipher_text

def decrypt(cipher_text, key):
    """
    Hill Cipher Decryption 
    p = C*K(Inv) MOD 26
    where
    P = plain text
    C = cipher text
    K = Key 
    K(INV) = Inverse of K
    """
    # encontrar o tamanho da matriz de acordo com o tamanho da chave
    matrix_size = findMatrixSize(len(key))
    # adicionar padding à chave e converter letras para números de acordo com a CHAR_TABLE
    padded_key = getPaddedKey(key, matrix_size) 
    # transformar a string da chave num array e depois numa matriz
    key_matrix = np.fromstring(padded_key,dtype=int, sep=' ').reshape(matrix_size,matrix_size)
    
    # converter o texto cifrado em números de acordo com o CHAR_TABLE 
    encoded_ct_string = encode(cipher_text)
    # transformar o texto cifrado já convertido em números num array e depois numa matriz
    cipher_text_matrix = np.fromstring(encoded_ct_string, dtype=int, sep=' ').reshape(-1, matrix_size)
    
    matrix = Matrix(key_matrix)
    # usa-se a função inv_mod do package sympy para calcular a matriz chave inversa (para decifrar)
    # a matriz é calculada da seguinte forma:
    '''
    (inv * I) MOD 26
    onde
    inv é a matriz inversa da matriz chave (key_matrix)
    I é o inverso modular multiplicativo da matriz chave

    o inverso modular multiplicativo é um valor tal que:
    (det(key_matrix) * I) MOD 26 = 1
    '''
    print(matrix)
    inverted_key_matrix = np.array(matrix.inv_mod(26)).astype(np.int32)

    plain_text = ''
    for column in cipher_text_matrix:
        # fazer a multiplicação da matriz chave inversa pelas colunas do texto cifrado seguido do calculo do resto da divisão por 26
        x = (inverted_key_matrix @ column) % 26

        for elem in x:
            # para cada elemento resultante do calculo anterior, converter o número em letra de acordo com o CHAR_TABLE_REV
            plain_text += CHAR_TABLE_REV[elem]
        # aqui tem-se o texto decifrado
    return plain_text



# funções auxiliares
def sanitize(text):
    return ''.join(c for c in text if c.isalpha()).lower()


def findMatrixSize(length):
    """
    encontrar o tamanho da matriz de acordo com o tamanho da chave (length)
    """
    start, size = 1, 2
    while True:
        if length in range(start, (size*size)+1):
            return size
        start = (size*size)+1
        size = size+1


def encode(string):
    """
    Returns space seperated encoded string 
    encoded string: string of numbers based of char table
    """
    return ' '.join([str(CHAR_TABLE[c]) for c in string])


def decode(matrix):
    """
    Returns string decoded from encoded matrix
    """
    text = ''
    for row, col in np.ndindex(matrix.shape):
        text += CHAR_TABLE_REV[matrix[row, col]]
    return text


def getPaddedKey(string, k):
    """
    adicionar padding à chave
    """
    for i in range(0, (k*k) - len(string)):
        string = string + list(CHAR_TABLE.keys())[i]
    return encode(string)


def getPaddedPlainText(string, k):
    """
    adicionar padding ao texto limpo
    apenas se necessário
    ou seja, se o tamanho do texto não for compativel com o da matriz
    """
    if len(string) % k != 0:
        if len(string) < k*k:
            #closest multiple  of k 
            for i in range(0,len(string) // k):
                string = string + list(CHAR_TABLE.keys())[i]
        elif len(string) > k*k:
            #compute expected size
            i = len(string)
            while i % k != 0:
                i+=1
            for i in range(0, i - len(string)):
                string = string + list(CHAR_TABLE.keys())[i]
    return encode(string)


if __name__ == "__main__":
    program()

#https://gist.github.com/avi-arora/4e91e6625a300f7f0a2ad488a5049edd

### Pergunta 4.1



Apesar das suas provas teóricas de segurança, a cifra "one-time pad" tem limitações sérias a nível prático.
Em primeiro lugar, em relação à segurança do processo de geração e troca da chave cifrada ("one time pad") que, conforme se sabe, deve ser tão longa como a mensagem. Assim, existem alguns problemas a si inerentes, o que faz com que facilmente se consigam levantar algumas questões.
 O principal fator prende-se com o facto da cifragem só ser tão segura quanto maior for a segurança associada à troca da chave.
Em segundo lugar é necessário um mecanismo de software que desenvolva "one-time pads" perfeitamente aleatórios. A nível prático este é um problema que não é nada trivial, visto que neste campo existem fatores que refletem a impossibilidade de garantir que as chaves geradas são únicas, facilmente comprometendo desde logo uma das principais características deste tipo de cifragem - de que cada chave tem que ser única.
Finalmente, é necessário um tratamento cuidadoso para ter a certeza que a chave mantém-se secreta/indecifrável para qualquer adversário. Por isso, espera-se que esta seja descartada de forma segura, prevenindo que haja reutilização total ou parcial da chave.

### Pergunta 5.1

Texto para cifrar: 'este exemplo mostra a transposição dupla' <br>
1ª chave: 'MINHO' -> ordem: 3 2 4 1 5 <br>
2ª chave: 'ENGSEG' -> ordem: 1 5 3 6 2 4 <br>
<br>
M I N H O &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; E N G S E G<br>
3 2 4 1 5 &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; 1 5 3 6 2 4<br>
<br>
E S T E E &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; E P S T P A<br>
X E M P L &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; L S E M A N<br>
O M O S T &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; I U E X O R<br>
R A A T R &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; A S D T M O<br>
A N S P O &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; A S C P E L<br>
S I C A O &ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp; T R O O A .<br>
D U P L A    

EPSTPAL SEMANIU EXORASD TMOASCP ELTROOA&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;ELIAAT PAOMEA SEEDCO ANROL. PSUSSR TMXTPO<br>
<br>
Texto cifrado final: 'eliaatpaomeaseedcoanrol.psussrtmxtpo'<br>
<br>
<br>
NOTA: Para facilitar a adaptação no código para a decifra, optou-se por usar um '.' nas células vazias da matriz.

In [ ]:
import numpy as np
import unidecode

def get_order(key):
    """
    função que dada uma chave retorna a ordem das colunas da matriz
    """
    abc = [chr(i+97) for i in range(0,26)] # lista com as letras do alfabeto
    order = [0 for _ in range(len(key))]

    i = 1
    for letter in abc:
        indices = [ind for ind, x in enumerate(key) if x == letter] # ver os indices onde letter aparece na chave
        # guardar no array order, no indice da letra da chave em questão o valor correspondente da ordem
        if len(indices) == 1:
            order[key.index(letter)] = i
            i+=1
        elif len(indices) > 1:
            for indice in indices:
                order[indice] = i
                i+=1
    return order


def encrypt(plain_text,key):
    """
    cifrar uma mensagem
    """
    # limpar o texto
    plain_text = ''.join(c for c in plain_text if c.isalpha()).upper()
    plain_text = unidecode.unidecode(plain_text)

    # calcular a ordem das colunas de acordo com a chave de cifra
    order = get_order(key=key)
    key_size = len(key)
    text_size = len(plain_text)
    text_array = []
    # transformar o texto num array
    for letter in plain_text:
        text_array.append(letter)

    # calcular o número de linhas necessárias para a matriz
    no_of_lines = int(text_size / key_size) + (text_size % key_size > 0)

    # criar a matriz com a mensagem a cifrar
    matrix = np.pad(text_array, (0, no_of_lines*key_size - len(text_array)), 
        mode='constant', constant_values='.').reshape(no_of_lines,key_size)

    ciphered_matrix = []
    # ler cada coluna da matriz, pela ordem calculada anteriormente
    for i in range(1,key_size+1):
        ciphered_matrix = np.append(ciphered_matrix,[row[order.index(i)] for row in matrix])

    # construir a mensagem cifrada resultante da leitura das colunas
    return ''.join(elem for sub in ciphered_matrix for elem in sub)


def decrypt(ciphered_text,key):
    """
    decifrar uma mensagem
    """
    # verificar se o texto cifrado contém alguma célula da matriz não preenchida (representada por '.' na string)
    if(ciphered_text[-1] == '.'):
        ciphered_text = ''.join(c for c in ciphered_text if c.isalpha()).upper()

    # calcular a ordem das colunas de acordo com a chave de decifra 
    order = get_order(key=key)
    text_size = len(ciphered_text)
    key_size = len(key)
    text_array = []
    # transformar o texto num array
    for letter in ciphered_text:
        text_array.append(letter)

    # calcular o número de linhas necessárias para a matriz 
    no_of_lines = int(text_size / key_size) + (text_size % key_size > 0)

    array = []
    # construir um array para transformar na matriz de onde resultou o texto cifrado, ou seja,
    # o texto pode ser dividido em blocos de 'no_of_lines' letras,
    # por cada número no array de ordem, ir ao "bloco" correspondente 
    # do texto cifrado e adicioná-lo na matriz
    # por exemplo, se o primeiro número no array ordem for 3, 
    # j irá iterar sob 'no_of_lines' letras a partir da posição
    # (3-1)*no_of_lines, que são os "blocos" de letras anteriores ao "bloco" 3
    for i in order:
        for j in range(no_of_lines):
            array.append(text_array[(i-1)*no_of_lines+j])   


    # reshape do array para uma matriz de 'key_size'*'no_of_lines'
    # seria de esperar que fosse uma matriz de 'no_of_lines'*'key_size'
    # mas decidiu-se construir desta forma para facilitar a posterior leitura 
    # assim, consideram-se que as linhas desta matriz são, na verdade, as colunas 
    # da matriz que teria sido usada para a cifra
    matrix = np.pad(matrix, (0, no_of_lines*key_size - len(text_array)), 
        mode='constant', constant_values='.').reshape(key_size,no_of_lines)

    string = ''
    # construir a mensagem decifrada resultante
    for j in range(no_of_lines):
        for i in range(key_size):
            string = string + (matrix[i][j])

    return string

def main():
    #key_1 = input('enter fst key: ')
    key_1 = 'minho'
    #key_2 = input('enter snd key: ')
    key_2 = 'engseg'

    #plain_text = input('enter text: ')
    plain_text = 'este exemplo mostra a transposição dupla'

    # cifra com a primeira chave
    fst_ciphered_text = encrypt(plain_text=plain_text,key=key_1)
    # cifra com a segunda chave
    final_ciphered_text = encrypt(plain_text=fst_ciphered_text,key=key_2)
    print('Encrypted text:', final_ciphered_text)
    
    # decifra com a segunda chave
    fst_deciphered_text = decrypt(ciphered_text=final_ciphered_text,key=key_2)
    # decifra com a primeira chave
    final_deciphered_text = decrypt(ciphered_text=fst_deciphered_text,key=key_1)
    print('Decrypted text:',final_deciphered_text)



if __name__== "__main__" :
    main()